# Finetune PE dataset for LI

## Libraries

In [1]:
# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [1]:
import os
import ast
import json
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report

In [4]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Load Dataset

In [5]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [ ]:
# *** MODEL NAME ***

# base_model = "unsloth/llama-3-8b-Instruct-bnb-4bit"

with open("tmp.pkl", "rb") as fh:
        
        l = pickle.load(fh)
        base_model = l[0]
        train_dataset_name = l[1]
        test_dataset_name = l[2]

In [6]:
dataset_dir = os.path.join(os.getcwd(), "datasets")

# *** TRAIN DATASET NAME *** #

# train_dataset_name = "PE_LI_train.json"
train_dataset_file = os.path.join(dataset_dir, train_dataset_name)

# *** TEST DATASET NAME *** #

# test_dataset_name = "PE_LI_test.json"
test_dataset_file = os.path.join(dataset_dir, test_dataset_name)

## Fine-tune Model

In [8]:
# *** MODEL DIR ***
model_name = f"""{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}"""

train_file = os.path.join(os.getcwd(), f"""cli_files/{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}.json""")
output_dir = os.path.join(os.getcwd(), "models", model_name)

nb_epochs = 10 # 3

In [9]:
train_file

'/Utilisateurs/umushtaq/cli_files/PE_LI_llama-3-70b-Instruct-bnb-4bit.json'

In [10]:
os.path.isfile(train_file)

True

In [ ]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
 }

In [12]:
dataset_info_line

{'file_name': '/Utilisateurs/umushtaq/datasets/PE_LI_train.json',
 'columns': {'prompt': 'instruction', 'query': 'input', 'response': 'output'}}

In [13]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [14]:
# with open("LLaMA-Factory/data/dataset_info.json", 'r+') as fh:
#     file_data = json.load(fh)
#     file_data["persuasive_essays"] = dataset_info_line
#     fh.seek(0)
#     json.dump(file_data, fh, indent = 4)

In [ ]:
with open("LLaMA-Factory/data/dataset_info.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["persuasive_essays"] = dataset_info_line

with open("LLaMA-Factory/data/dataset_info.json", "w") as jsonFile:
    json.dump(data, jsonFile)

In [15]:
args = dict(
  stage="sft",                           # do supervised fine-tuning
  do_train=True,
  model_name_or_path=base_model,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="persuasive_essays",           # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir=output_dir,                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=3000,                       # save checkpoint every 1000 steps
  learning_rate=5e-5,                    # the learning rate
  num_train_epochs=nb_epochs,            # the epochs of training
  max_samples=2000,                       # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                    # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  fp16=True,                             # use float16 mixed precision training
  report_to="none"                       # discards wandb
)

In [16]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [17]:
os.getcwd()

'/Utilisateurs/umushtaq'

In [18]:
%cd LLaMA-Factory/
!set train_file = train_file

/Utilisateurs/umushtaq/LLaMA-Factory


/Utilisateurs/umushtaq/.conda/envs/llama-factory-notebook/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [19]:
train_file

'/Utilisateurs/umushtaq/cli_files/PE_LI_llama-3-70b-Instruct-bnb-4bit.json'

In [20]:
!llamafactory-cli train $train_file

06/05/2024 10:36:38 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
06/05/2024 10:36:38 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-06-05 10:36:38,226 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-06-05 10:36:38,227 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-06-05 10:36:38,227 >> loading file special_tokens_map.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-06-0

## Inference on the fine-tuned model

In [21]:
output_dir

'/Utilisateurs/umushtaq/models/PE_LI_llama-3-70b-Instruct-bnb-4bit'

In [22]:
os.listdir(output_dir)

['.ipynb_checkpoints',
 'trainer_log.jsonl',
 'README.md',
 'adapter_model.safetensors',
 'adapter_config.json',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'tokenizer.json',
 'training_args.bin',
 'train_results.json',
 'all_results.json',
 'trainer_state.json']

In [23]:
# %cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path=base_model, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=output_dir,            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)


In [24]:
model = ChatModel(args)

[INFO|tokenization_utils_base.py:2108] 2024-06-05 14:44:20,376 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-06-05 14:44:20,379 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-06-05 14:44:20,381 >> loading file special_tokens_map.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-06-05 14:44:20,383 >> loading file tokenizer_config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/tokenizer_config.json
[WARNING|logging.py:314] 2024-06-05 14:44:20,730 >>

06/05/2024 14:44:20 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


/Utilisateurs/umushtaq/.conda/envs/llama-factory-notebook/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-06-05 14:44:20,884 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/config.json
[INFO|configuration_utils.py:796] 2024-06-05 14:44:20,887 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-70b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_siz

06/05/2024 14:44:20 - INFO - llamafactory.model.utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
06/05/2024 14:44:20 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:393] 2024-06-05 14:44:24,328 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3474] 2024-06-05 14:44:24,392 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/model.safetensors.index.json
[INFO|modeling_utils.py:1519] 2024-06-05 14:44:24,529 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:962] 2024-06-05 14:44:24,532 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009
}



Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4280] 2024-06-05 14:55:52,245 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4288] 2024-06-05 14:55:52,247 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at unsloth/llama-3-70b-Instruct-bnb-4bit.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:917] 2024-06-05 14:55:52,451 >> loading configuration file generation_config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-70b-Instruct-bnb-4bit/snapshots/df8fab159e49c133bb4f074baeff4b303614b6d1/generation_config.json
[INFO|configuration_utils.py:962] 2024-06-05 14:55:52,454 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128009
  ]
}



06/05/2024 14:55:52 - INFO - llamafactory.model.utils.attention - Using torch SDPA for faster training and inference.
06/05/2024 14:55:52 - INFO - llamafactory.model.adapter - Upcasting trainable params to float32.
06/05/2024 14:55:52 - INFO - llamafactory.model.adapter - Fine-tuning method: LoRA
06/05/2024 14:56:04 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/models/PE_LI_llama-3-70b-Instruct-bnb-4bit
06/05/2024 14:56:04 - INFO - llamafactory.model.loader - all params: 70657253376


In [25]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

In [26]:
test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append("\nUser:" + sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [27]:
test_predictions = []

for prompt in test_prompts:

    messages = []
    messages.append({"role": "user", "content": prompt})

    response = ""
    
    for new_text in model.stream_chat(messages):
        #print(new_text, end="", flush=True)
        response += new_text
        #print()
    test_predictions.append({"role": "assistant", "content": response})

    torch_gc()

In [31]:
with open(os.path.join(output_dir, f"""PE_LI_results_{nb_epochs}.pickle"""), 'wb') as fh:
    results_d = {"ground_truths": test_grounds,
                 "predictions": test_predictions    
        
    }
    pickle.dump(results_d, fh)

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()